In [21]:
from pathlib import Path
from sqlalchemy import create_engine
from langchain.sql_database import SQLDatabase
import sqlite3
from langchain.chains import create_sql_query_chain
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

from dotenv import load_dotenv
import os
import time
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



load_dotenv()

#Langsmith Tracking
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACKING_V2']="true"
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')
groq_api_key=os.getenv('GROQ_API_KEY')

llm=ChatGroq(model='llama3-8b-8192', groq_api_key=groq_api_key)

def configure_db():
    
        dbfilepath=(Path("C:/Users/Dhairya Arora/Videos/Gen AI/LANGCHAIN/aisha/walmart.db")).absolute()
        print(dbfilepath)
        creator=lambda: sqlite3.connect(f"file:{dbfilepath}?mode=rwc",uri=True)
        return SQLDatabase(create_engine("sqlite:///", creator=creator))
    
db=configure_db()


# chainSec = create_sql_query_chain(model, db)

# toolkit=SQLDatabaseToolkit(db=db,llm=model)

# agent=create_sql_agent(
# prompt=prompt,
# llm=model,
# toolkit=toolkit,
# verbose=True,
# agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
# )

# agent.run({
#         'messages': [ HumanMessage('Find records of userId 1')]
# })


C:\Users\Dhairya Arora\Videos\Gen AI\LANGCHAIN\aisha\walmart.db


In [22]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools


[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001DDA64AE690>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001DDA64AE690>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001DDA64AE690>),
 QuerySQLCheckerTool(description='Use this tool to 

In [23]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """
You are an agent designed to interact with a SQL database.
If you see a name or a phone number in query extract it and fetch the details of the customer from the customer table,
match the firtname and lastname of the customer with the name in the query or phone number.
ignore rest of the query. If you see an orderId in the query, extract it and fetch the details of the order from the database, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 8 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [24]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model=llm, tools=tools, messages_modifier=system_message)

C:\Users\Dhairya Arora\AppData\Local\Temp\ipykernel_25912\446365532.py:4: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.2.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(model=llm, tools=tools, messages_modifier=system_message)


In [28]:
# Initialize a variable to store the final result
final_output = None

# Stream through the responses
for s in agent_executor.stream(
    {
        'messages': [HumanMessage('My firstname is Dhairya Arora and my phone number is 9811264318. Tell me my address from the database')]
    }
):
    # Check if the message is from the agent
    if 'agent' in s and 'messages' in s['agent']:
        # Loop through the messages to find the final AIMessage
        for message in s['agent']['messages']:
            if isinstance(message, AIMessage):
                final_output = message.content

# Print or use the final output
if final_output:
    print("Final Output:")
    print(final_output)
else:
    print("No final output found.")


Exception closing connection <sqlite3.Connection object at 0x000001DDA71197B0>
Traceback (most recent call last):
  File "c:\Users\Dhairya Arora\Videos\Gen AI\LANGCHAIN\venv\Lib\site-packages\sqlalchemy\pool\base.py", line 376, in _close_connection
    self._dialect.do_close(connection)
  File "c:\Users\Dhairya Arora\Videos\Gen AI\LANGCHAIN\venv\Lib\site-packages\sqlalchemy\engine\default.py", line 707, in do_close
    dbapi_connection.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 26748 and this is thread id 9172.
Exception closing connection <sqlite3.Connection object at 0x000001DDA71193F0>
Traceback (most recent call last):
  File "c:\Users\Dhairya Arora\Videos\Gen AI\LANGCHAIN\venv\Lib\site-packages\sqlalchemy\pool\base.py", line 376, in _close_connection
    self._dialect.do_close(connection)
  File "c:\Users\Dhairya Arora\Videos\Gen AI\LANGCHAIN\venv\Lib\site-packages\sqlalchemy\engine

Final Output:
<tool-use></tool-use>
